In [8]:
import numpy as np
import metnum
from scoring import cross_validation as cv
from scoring import metrics

In [9]:
import pandas as pd

df_train = pd.read_csv("../data/train.csv")
df_train.sample(frac=1)
#df_train = df_train[:5000]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
32925,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8263,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30094,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32475,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7904,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21383,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31184,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36051,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36265,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

In [17]:
%%time
alpha = 20
n_iter = 5000
eps = 1e-2

pca = metnum.PCA(alpha)
X = pca.transform(X, n_iter, eps)

CPU times: user 22 µs, sys: 1e+03 ns, total: 23 µs
Wall time: 26 µs


In [19]:
limit = int(0.8 * X.shape[0])

X_train, y_train = X[:limit], y[:limit]
X_val, y_val = X[limit:], y[limit:]

In [20]:
clf = metnum.KNNClassifier(10)
clf.fit(X_train, y_train)

In [21]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_val)

acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.05095238095238095
CPU times: user 24.5 s, sys: 98.8 ms, total: 24.6 s
Wall time: 24.5 s


## Cross validation ##

In [17]:
%%time
import os
import numpy as np

class MatrixFSDict:
    """Mantiene un FS dict de matrices de numpy"""

    def __init__(self, d):
        self.dir = d

    def load(self, name: str) -> np.ndarray:
        path = os.path.join(self.dir, name)
        print("loading:", path)
        return np.loadtxt(path, delimiter=',')
    
    def store(self, name: str, X: np.ndarray):
        path = os.path.join(self.dir, name)
        print("storing:", path)
        return np.savetxt(path, X, delimiter=",")
    
    def contains(self, name: str):
        os.path.exists(os.path.join(self.dir, name))


storing: pca-cache/fold_1-alpha_100.csv
CPU times: user 10.8 ms, sys: 111 ms, total: 122 ms
Wall time: 125 ms


In [12]:
DIR = "pca-cache"
MAX_ALPHA = 100

class CachedPCA:
    def __init__(self, alpha: int):
        assert(alpha < MAX_ALPHA)
        self.pca = metnum.PCA(alpha)
        self.cache = MatrixFSDict(DIR)

    def fit(X, y, fold: int):
        name = "fold_{}".format(fold)
        if cache.contains(name):
            # HIT!
            tc = cache.load(name)
            # verificar!!!!!!!!!!!
            self.pca.set_tc(tc[:self.alpha+1])
            return
        
        # MISS :(
        max_pca = metnum.PCA(MAX_ALPHA)
        max_pca.fit(X)
        tc = max_pca.get_tc()
        cache.save(name, tc)
    
        self.pca.set_tc(tc[:self.alpha+1])
        
    def transform(X):
        return self.pca.transform(X)

class KNNnPCAClassifier:
    def __init__(self, k, alpha):
        self.knn_classifier = metnum.KNNClassifier(k)
        self.pca = CachedPCA(alpha)

    def fit(self, X_train, y_train, fold=None):
        self.pca.fit(...)
        self.knn_classifier.fit(self.pca.transform(X_train, 5000, 1e-2), y_train)
    
    def predict(self, X):
        return self.knn_classifier.predict(self.pca.transform(X, 5000, 1e-2))

loading: pca-cache/fold_1-alpha_100.csv


ValueError: could not convert string to float: '1.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00'

In [ ]:
%%time
clf = KNNnPCAClassifier(10,20)
mean = cv.cross_validate(clf, X, y, metrics.accuracy_score, 4, True)

In [7]:
clf = KNNnPCAClassifier(10,20)
clf.predict(X, **{"l_bound": 2, "r_bound": 1})

TypeError: predict() got an unexpected keyword argument 'l_bound'

In [10]:
mean

NameError: name 'mean' is not defined